<a href="https://colab.research.google.com/github/goffycoder/paraphrase/blob/main/Bart_large_training1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Installing the necessary libraries

In [ ]:
!pip install transformers
!pip install datasets
!pip install rouge-score

# Step 2: Importing the necessary libraries


In [ ]:
import numpy as np
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
from rouge_score import rouge_scorer
import nltk

# Step 3: Loading and preprocessing the dataset


Loading the dataset

In [ ]:
dataset = load_dataset("HHousen/ParaSCI")


  0%|          | 0/3 [00:00<?, ?it/s]

Preprocessing the dataset & attention mask


In [ ]:
def preprocess(example):
    return {'input_ids': tokenizer(example['sentence1'], truncation=True, padding='max_length', max_length=1024)['input_ids'],
            'labels': tokenizer(example['sentence2'], truncation=True, padding='max_length', max_length=1024)['input_ids'],
            'attention_mask': tokenizer(example['sentence1'], truncation=True, padding='max_length', max_length=1024)['attention_mask']}


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Defining the tokenizer

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')


Preprocessing the dataset

In [ ]:
dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/4894 [00:00<?, ? examples/s]

Showing the first few preprocessed sentences

In [ ]:
for i in range(5):
    print("Original Sentence: ", tokenizer.decode(dataset['train'][i]['input_ids']))
    print("Paraphrase: ", tokenizer.decode(dataset['train'][i]['labels']))
    print("\n")


Original Sentence:  <s>for all methods, the tweets were tokenized with the cmu twitter nlp tool.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

# Step 4: Splitting dataset in training, validation & test sets


In [ ]:
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']


# Step 5: Defining the compute metrics function


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


Defining the compute_metrics function

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replacing -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = scorer.score(decoded_labels, decoded_preds)
    # Extracting a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Adding mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 6: Defining the model, training arguments & trainer


In [ ]:
# Defining the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

Defining the training arguments


In [ ]:
!pip install accelerate -U


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)


Defining the trainer


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # validation dataset
    compute_metrics=compute_metrics,     # the compute_metrics function
)


# Step 7: Training and evaluating the model


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
# Training the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

Evaluating the model on the test dataset


In [ ]:
trainer.evaluate(test_dataset)